# Getting Started with Tools and Agents in Mirascope

This notebook provides a detailed introduction to using Tools and implementing Agents in Mirascope. We'll use the WebSearchAgent as our primary example to demonstrate these concepts.

1. [Introduction](#Introduction)
2. [Setting Up the Environment](#Setting-Up-the-Environment)
3. [Understanding Tools in Mirascope](#Understanding-Tools-in-Mirascope)
4. [Creating Custom Tools](#Creating-Custom-Tools)
5. [Introduction to Agents](#Introduction-to-Agents)
6. [Implementing the WebSearchAgent](#Implementing-the-WebSearchAgent)
7. [Running and Testing the Agent](#Running-and-Testing-the-Agent)
8. [Advanced Concepts and Best Practices](#Advanced-Concepts-and-Best-Practices)
9. [Conclusion](#Conclusion)

## 1. Introduction

Tools and Agents are two key concepts in building advanced AI systems, particularly those involving Large Language Models (LLMs):

- **Tools**: Functions that extend the capabilities of LLMs, allowing them to perform specific tasks or access external data.
- **Agents**: Autonomous or semi-autonomous entities that use LLMs and Tools to perform complex tasks or interact with users.

In this notebook, we'll explore how to create and use Tools, and how to implement an Agent using the WebSearchAgent as our example. We'll be using Mirascope, a framework that simplifies the process of building LLM-powered applications with tools and agents.

For more detailed information on these concepts, refer to the following Mirascope documentation:

- [Tools documentation](https://docs.mirascope.io/latest/learn/tools/)
- [Agents documentation](https://docs.mirascope.io/latest/learn/agents/)

## 2. Setting Up the Environment

First, let's set up our environment by installing Mirascope and other necessary libraries.

In [ ]:
!pip install "mirascope[openai]" duckduckgo_search beautifulsoup4 requests

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

For more information on setting up Mirascope and its dependencies, see the [Mirascope installation guide](https://docs.mirascope.io/latest/get-started/).

## 3. Understanding Tools in Mirascope

In Mirascope, Tools are functions that extend the capabilities of LLMs. They allow the LLM to perform specific tasks or access external data. Tools are typically used to:

1. Retrieve information from external sources
2. Perform calculations or data processing
3. Interact with APIs or databases
4. Execute specific actions based on the LLM's decisions

Let's start by creating a simple Tool that extracts content from a webpage:

In [1]:
import re

import requests
from bs4 import BeautifulSoup


def extract_content(url: str) -> str:
    """Extract the main content from a webpage.

    Args:
        url: The URL of the webpage to extract the content from.

    Returns:
        The extracted content as a string.
    """
    try:
        response = requests.get(url, timeout=5)

        soup = BeautifulSoup(response.content, "html.parser")

        unwanted_tags = ["script", "style", "nav", "header", "footer", "aside"]
        for tag in unwanted_tags:
            for element in soup.find_all(tag):
                element.decompose()

        main_content = (
            soup.find("main")
            or soup.find("article")
            or soup.find("div", class_=re.compile("content|main"))
        )

        if main_content:
            text = main_content.get_text(separator="\n", strip=True)
        else:
            text = soup.get_text(separator="\n", strip=True)

        lines = (line.strip() for line in text.splitlines())
        return "\n".join(line for line in lines if line)
    except Exception as e:
        return f"{type(e)}: Failed to extract content from URL {url}"

This `extract_content` function is a Tool that takes a URL as input and returns the main content of the webpage as a string. It uses BeautifulSoup to parse the HTML and extract the relevant text content.

For more details on implementing and using Tools in Mirascope, see the [Tools documentation](https://docs.mirascope.io/latest/learn/tools/).

## 4. Creating Custom Tools

Now, let's create a more complex Tool that performs web searches using the DuckDuckGo search engine:

In [1]:
from duckduckgo_search import DDGS


def web_search(queries: list[str], max_results_per_query: int = 2) -> str:
    """Performs web searches for given queries and returns URLs.

    Args:
        queries: List of search queries.
        max_results_per_query: Maximum number of results to return per query.

    Returns:
        str: Newline-separated URLs from search results or error messages.

    Raises:
        Exception: If web search fails entirely.
    """
    try:
        urls = []
        for query in queries:
            results = DDGS(proxies=None).text(query, max_results=max_results_per_query)

            for result in results:
                link = result["href"]
                try:
                    urls.append(link)
                except Exception as e:
                    urls.append(f"{type(e)}: Failed to parse content from URL {link}")
        return "\n\n".join(urls)

    except Exception as e:
        return f"{type(e)}: Failed to search the web for text"

This `web_search` function is a more complex Tool that takes a list of search queries and returns a string of newline-separated URLs from the search results. It uses the DuckDuckGo search API to perform the searches.

## 5. Introduction to Agents

Agents in Mirascope are autonomous or semi-autonomous entities that use LLMs and Tools to perform complex tasks or interact with users. They typically have:

1. A state (e.g., conversation history, search history)
2. Access to one or more Tools
3. A method for interacting with an LLM
4. A main loop or execution flow

Let's start implementing our WebSearchAgent:

In [1]:
from pydantic import BaseModel

from mirascope.core import BaseMessageParam, openai


class WebAssistantBase(BaseModel):
    messages: list[BaseMessageParam | openai.OpenAIMessageParam] = []
    search_history: list[str] = []
    max_results_per_query: int = 2

    def _web_search(self, queries: list[str]) -> str:
        """Wrapper for the web_search function that updates search history."""
        result = web_search(queries, self.max_results_per_query)
        self.search_history.extend(queries)
        return result

This `WebAssistant` class forms the base of our Agent. It has:

- A `messages` list to store the conversation history
- A `search_history` list to keep track of past searches
- A `max_results_per_query` parameter to control the number of search results
- A `_web_search` method that wraps our `web_search` Tool and updates the search history

To learn more about the concept of Agents in Mirascope, refer to the [Agents documentation](https://docs.mirascope.io/latest/learn/agents/).

## 6. Implementing the WebSearchAgent

Now, let's add the LLM interaction and main execution flow to our WebSearchAgent:

In [1]:
from datetime import datetime

from mirascope.core import openai, prompt_template


class WebAssistant(WebAssistantBase):
    @openai.call(model="gpt-4o-mini", stream=True)
    @prompt_template(
        """
        SYSTEM:
        You are an expert web searcher. Your task is to answer the user's question using the provided tools.
        The current date is {current_date}.

        You have access to the following tools:
        - `_web_search`: Search the web when the user asks a question. Follow these steps for EVERY web search query:
            1. There is a previous search context: {self.search_history}
            2. There is the current user query: {question}
            3. Given the previous search context, generate multiple search queries that explores whether the new query might be related to or connected with the context of the current user query. 
                Even if the connection isn't immediately clear, consider how they might be related.
        - `extract_content`: Parse the content of a webpage.

        When calling the `_web_search` tool, the `body` is simply the body of the search
        result. You MUST then call the `extract_content` tool to get the actual content
        of the webpage. It is up to you to determine which search results to parse.

        Once you have gathered all of the information you need, generate a writeup that
        strikes the right balance between brevity and completeness based on the context of the user's query.

        MESSAGES: {self.messages}
        USER: {question}
        """
    )
    async def _stream(self, question: str) -> openai.OpenAIDynamicConfig:
        return {
            "tools": [self._web_search, extract_content],
            "computed_fields": {
                "current_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            },
        }

    async def _step(self, question: str):
        response = await self._stream(question)
        tools_and_outputs = []
        async for chunk, tool in response:
            if tool:
                print(f"using {tool._name()} tool with args: {tool.args}")
                tools_and_outputs.append((tool, tool.call()))
            else:
                print(chunk.content, end="", flush=True)
        if response.user_message_param:
            self.messages.append(response.user_message_param)
        self.messages.append(response.message_param)
        if tools_and_outputs:
            self.messages += response.tool_message_params(tools_and_outputs)
            await self._step("")

    async def run(self):
        while True:
            question = input("(User): ")
            if question == "exit":
                break
            print("(Assistant): ", end="", flush=True)
            await self._step(question)
            print()

This implementation includes:

1. The `_stream` method, which sets up the LLM call with the necessary tools and computed fields.
2. The `_step` method, which processes the LLM response, handles tool calls, and updates the conversation history.
3. The `run` method, which implements the main interaction loop for the agent.

The `@openai.call` and `@prompt_template` decorators are used to set up the LLM interaction and define the prompt for the agent.

For more information on creating custom Agents and advanced Agent patterns, see the [Agents documentation](https://docs.mirascope.io/latest/learn/agents/).

## 7. Running and Testing the Agent

Now that we have implemented our WebSearchAgent, let's run and test it:

In [1]:
async def main():
    web_assistant = WebAssistant()
    await web_assistant.run()


# Run main in a jupyter notebook
await main()

# Run main in a python script
# asyncio.run(main())

(Assistant): using _web_search tool with args: {'queries': ['what is a Large Language Model', 'applications of Large Language Models', 'how do Large Language Models work', 'limitations of Large Language Models', 'ethics of Large Language Models', 'recent advancements in Large Language Models']}
using extract_content tool with args: {'url': 'https://www.ibm.com/topics/large-language-models'}
using extract_content tool with args: {'url': 'https://en.wikipedia.org/wiki/Large_language_model'}
using extract_content tool with args: {'url': 'https://medium.com/@researchgraph/the-journey-of-large-language-models-evolution-application-and-limitations-c72461bf3a6f'}
using extract_content tool with args: {'url': 'https://blogs.nvidia.com/blog/what-are-large-language-models-used-for/'}
using extract_content tool with args: {'url': 'https://arstechnica.com/science/2023/07/a-jargon-free-explanation-of-how-ai-large-language-models-work/'}
### Overview of Large Language Models (LLMs)

Large Language M

To use the WebSearchAgent, run the code above and start interacting with it by typing your questions. The agent will use web searches and content extraction to provide answers. Type 'exit' to end the interaction.

## 8. Advanced Concepts and Best Practices

When working with Tools and Agents in Mirascope, consider the following best practices:

1. **Error Handling**: Implement robust error handling in your Tools and Agent implementation.
2. **Rate Limiting**: Be mindful of rate limits when using external APIs in your Tools.
3. **Caching**: Implement caching mechanisms to improve performance and reduce API calls.
4. **Testing**: Write unit tests for your Tools and integration tests for your Agents.
5. **Modularity**: Design your Tools and Agents to be modular and reusable.
6. **Security**: Be cautious about the information you expose through your Tools and Agents.
7. **Logging**: Implement logging to track the behavior and performance of your Agents.

For more advanced usage, you can explore concepts like:

- Multi-agent systems
- Tool chaining and composition
- Dynamic tool selection
- Memory and state management for long-running agents

For more advanced techniques and best practices in using Mirascope, refer to the following documentation:

- [Dynamic Configuration](https://docs.mirascope.io/latest/learn/dynamic_configuration/)
- [Chaining](https://docs.mirascope.io/latest/learn/chaining/)
- [Streams](https://docs.mirascope.io/latest/learn/streams/)

## 9. Conclusion

In this notebook, we've explored the basics of creating Tools and implementing Agents in Mirascope. We've built a WebSearchAgent that can perform web searches, extract content from webpages, and use an LLM to generate responses based on the gathered information.

This example demonstrates the power and flexibility of Mirascope in building AI applications that combine LLMs with custom tools and logic. As you continue to work with Mirascope, you'll discover more advanced features and patterns that can help you build even more sophisticated AI agents and applications.